In [1]:
import torch 
from torch import nn
from torch.nn import functional as F
import numpy as np
import pandas as pd
from vocab import Vocab
from dataprocessor import DataProcessor
from conv_model import ConvProtein

In [2]:
def get_data(filename):
    data = pd.read_csv(filename)
    return data['protein_sequence'].values , data['tm'].values 

trainx,trainy = get_data('./data/train_fixed.csv')
vocab = Vocab()
vocab.from_file()




tensor([[[[17.,  1.,  7.,  ...,  2., 18., 11.],
          [ 1.,  2.,  4.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          ...,
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]]],


        [[[17.,  3.,  3.,  ..., 12.,  5., 12.],
          [ 8.,  3.,  2.,  ...,  9.,  2.,  7.],
          [ 4.,  6.,  9.,  ..., 18., 18.,  3.],
          ...,
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]]],


        [[[17.,  2.,  4.,  ...,  5.,  1.,  8.],
          [12., 16., 14.,  ...,  9., 18., 16.],
          [ 7.,  1.,  6.,  ...,  0.,  0.,  0.],
          ...,
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]]],


        ...,


        [[[17.,  2.,  4.,  ...,  5., 11., 18.],
          [19., 

In [ ]:
model = ConvProtein([21,32])

inputs = torch.randint(1,21,size=[13,60,60])

out = model(inputs)

print(out)